# Software 0.1 Demo
### CS488 - March 2020 - Porter Libby 
My environment includes the imports listed below in Python 3.7.5.


In [37]:
# ---------- IMPORTS ----------

%reload_ext autoreload
%autoreload 2

from PIL import Image
from fastai.vision import *
from matplotlib import pyplot as plt
import numpy as np
from scipy import ndimage as ndi
import argparse
import glob
import cv2
import time
import os
import sys
import shutil
from IPython.display import clear_output

In [38]:
# progress bars
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [39]:
# ---------- CHANNEL COMPRESSION ----------

def createMultiChannelImage(fpArr):
    ''' Open multiple images and return a single multi channel image '''
    mat = None
    nChannels = len(fpArr)
    for i,fp in enumerate(fpArr):
        #print('Loading: ', fp)
        img = PIL.Image.open(fp)
        chan = pil2tensor(img, np.float32).float().div_(255)
        if(mat is None):
            mat = torch.zeros((nChannels,chan.shape[1],chan.shape[2]))
        mat[i,:,:]=chan
    return Image(mat)

In [40]:
# ---------- IMAGE PREPROCESSOR ----------

def image_preprocess(in_path):
    """ Takes a directory path, returns three base versions of image. """
    image = cv2.imread(in_path) 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    return blurred

In [41]:
# ---------- EDGE OPERATORS ----------

# --- OpenCV2
def canny_auto(in_path, sigma=0.33):
    #print("Creating canny_auto...")
    img = image_preprocess(in_path)
    v = np.median(img)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    cannyauto = cv2.Canny(img, lower, upper)
    cv2.imwrite('tmp/canny_auto.jpg', cannyauto)
    return 'tmp/canny_auto.jpg'
def canny_wide(in_path):
    """ Take a directory path, writes result to another path """
    #print("Creating canny_wide...")
    img = image_preprocess(in_path)
    cannywide = cv2.Canny(img, 10, 200)
    cv2.imwrite('tmp/canny_wide.jpg', cannywide)
    return 'tmp/canny_wide.jpg'
def canny_tight(in_path):
    """ Take a directory path, writes result to another path """
    #print("Creating canny_tight...")
    img = image_preprocess(in_path)
    cannytight = cv2.Canny(img, 225, 250)
    cv2.imwrite('tmp/canny_tight.jpg', cannytight)
    return 'tmp/canny_tight.jpg'
def laplacian(in_path):
    """ Take a directory path, writes result to another path """
    #print("Creating laplacian...")
    img = image_preprocess(in_path)
    lap = cv2.Laplacian(img,cv2.CV_64F)
    cv2.imwrite('tmp/laplacian.jpg', lap)
    return 'tmp/laplacian.jpg'
def sobel_x(in_path):
    """ Take a directory path, writes result to another path """
    #print("Creating sobel_x...")
    img = image_preprocess(in_path)
    sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)  # x
    cv2.imwrite('tmp/sobel_x.jpg', sobelx)
    return 'tmp/sobel_x.jpg'
def sobel_y(in_path):
    """ Take a directory path, writes result to another path """
    #print("Creating sobel_y...")
    img = image_preprocess(in_path)
    sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)  # y
    cv2.imwrite('tmp/sobel_y.jpg', sobely)
    return 'tmp/sobel_y.jpg'

# --- Scipy
def prewitt(in_path):
    """ Take a directory path, writes result to another path """
    #print("Creating prewitt...")
    img = image_preprocess(in_path)
    p = ndi.prewitt(img) 
    cv2.imwrite('tmp/prewitt.jpg', p)
    return 'tmp/prewitt.jpg'


In [45]:
# ---------- IMAGE PROCESSING HANDLERS ----------

def createSingleInput(path):
    img = createMultiChannelImage([
        prewitt(path),
        laplacian(path),
        canny_tight(path)
    ])
    img.save('out/COMP.jpg')
    print('Done!')
    return img

def createMultipleInput(path_ls,out_path):
    imgs = []
    for x in range(len(path_ls)):
        update_progress(x / len(path_ls))
        img = createMultiChannelImage([
            prewitt(path_ls[x]),
            canny_auto(path_ls[x]),
            sobel_x(path_ls[x])
        ])
        img.save(out_path+str(x)+'.jpg')
        imgs.append(img)
    print('Done!')
    return imgs

def absoluteFilePaths(directory):
    paths = []
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            paths.append(os.path.abspath(os.path.join(dirpath, f)))
    return paths
        
def preprocessDatabase(path): # top level folder as input
    subfolders = [ f.path for f in os.scandir(path) if f.is_dir() ]
    for label in subfolders:
        imgs = []
        for file in os.listdir(label):
            if file.endswith(".jpg"):
                imgs.append(os.path.join(label, file))
                
        dirpath = os.path.join("out", label.split('/')[-1])
        
        if os.path.exists(dirpath) and os.path.isdir(dirpath):
            shutil.rmtree(dirpath)
        os.mkdir(dirpath)
        
        createMultipleInput(imgs, "out/"+label.split('/')[-1]+"/")
        
    
        

In [46]:
# ---------- CONVOLUTIONAL NEURAL NETWORK ----------
def CreateModelFromPath(path, folder_stats=True, image_stats=True):  #
    size = 224
    bs = 4
    data = ImageDataBunch.from_folder(path, 
        ds_tfms=get_transforms(do_flip=True, flip_vert=True),
        valid_pct=0.5, 
        size=size, 
        bs=bs)
    
    if folder_stats:
        labels = os.listdir(path)
        print("No. of labels: {}".format(len(labels)))
        print("-----------------")
        for label in labels:
            print("{}, {} files".format(label, len(os.listdir(path+label))))

    if image_stats:
        print("\n Data to be run:")
        print("-----------------")
        data.normalize(imagenet_stats)
        data.show_batch(rows=3, figsize=(3,3))
        data.classes

    # Create CNN Learner
    learner = cnn_learner(data, models.resnet34, metrics=[accuracy])
    learner.fit_one_cycle(5,max_lr=1e-2)
    interp = ClassificationInterpretation.from_learner(learner)
    interp.plot_confusion_matrix(figsize=(3,3))

## DEMO - Pre-processing & Edge Detection

Take an input path and create a parallel set of same-name folders with the processed versions of images.

Settings are hard coded for this example.

Processed images will be a channel compression of the following operations:
- prewitt
- canny_auto
- sobel_x


In [ ]:
preprocessDatabase("../training/plant-id/")

Progress: [#############-------] 66.3%


## DEMO - Machine Learning

This step will create a resnet34 model based on the input path given.

Settings are hard coded for this example.

Model will do 5 learning cycles and then output confusion matrix.

In [ ]:
CreateModelFromPath("out", False, False)

epoch,train_loss,valid_loss,accuracy,time
0,1.054363,0.758447,0.751079,12:23
1,0.822289,0.456308,0.855626,12:19
2,0.518745,0.305763,0.882177,12:19
3,0.395875,0.281710,0.895121,12:18
